## Setup spark

### Setup python environments

In [2]:
# Add the vevn to spark's settings, so inject the venv’s Python into both driver & worker configs before recreating the session, to find the right python interpreter, since the notebook us running within the venv kernel, we can just use sys.executable
import os, sys

print(sys.executable)
venv_python = sys.executable

# 1) Ensure the worker uses exactly this Python executable:
os.environ['PYSPARK_PYTHON'] = venv_python
os.environ['PYSPARK_DRIVER_PYTHON'] = venv_python

print(os.environ['PYSPARK_PYTHON'])
print(os.environ['PYSPARK_DRIVER_PYTHON'])


c:\Sandeep SSD\Programming SSD\Data Engineering Zoomcamp\data-engineering-zoomcamp\dataenginzoomvenv\Scripts\python.exe
c:\Sandeep SSD\Programming SSD\Data Engineering Zoomcamp\data-engineering-zoomcamp\dataenginzoomvenv\Scripts\python.exe
c:\Sandeep SSD\Programming SSD\Data Engineering Zoomcamp\data-engineering-zoomcamp\dataenginzoomvenv\Scripts\python.exe


### Launch spark with those python variables

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.pyspark.python", venv_python) \
    .config("spark.pyspark.driver.python", venv_python) \
    .getOrCreate()

In [4]:
# Check which port the Spark UI is running on
print(spark.sparkContext.uiWebUrl)

http://192.168.0.181:4040


## Read the files

### Read green files

In [6]:
# Use 2 * since we have year subfolders, and month subfolders, okay...
df_green = spark.read.parquet('../../Data/data/csv/green/spark_parquet/*/*')

In [9]:
# registertemptable is deprecated 
# df_green.registerTempTable('green')
df_green.createOrReplaceTempView('green')

In [12]:
# Same query in notebook 6 but smaller
# Looks at total amount and sees how many records are there per zone (location id) per hour
# Checks only trips from after 2020
# count(1) same as count(*), just however you prefer to write it, count(*/1) will count the number of records for the group of (1, 2)
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
order by 
    1, 2
""")

In [13]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7|            769.73|            45|
|2020-01-01 00:00:00|  17|195.02999999999997|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|129.95999999999998|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|175.67000000000002|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [15]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('../../Data/data/module-05-batch/07-groupby-join/report/revenue/green', mode='overwrite')

### Read yellow files

In [18]:
df_yellow = spark.read.parquet('../../Data/data/csv/yellow/spark_parquet/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [19]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [ ]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../../Data/data/module-05-batch/07-groupby-join/report/revenue/yellow', mode='overwrite')

In [46]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [47]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [48]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [50]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [51]:
df_join = spark.read.parquet('data/report/revenue/total')

In [56]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [54]:
df_zones = spark.read.parquet('zones/')

In [57]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [62]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')